In [1]:
# Yernar Shambayev, DL-2
# Реализовать обучение линейной регрессии для задачи boston house prices
# с использованием torch’а

import torch
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('housing.csv', header=None, delimiter=r"\s+")
print(df.head())
features_data = df[df.columns[:-1]]
features_data = features_data.apply(lambda x: (x - x.mean()) / x.std()) # Нормализация

        0     1     2   3      4      5     6       7   8      9     10  \
0  0.00632  18.0  2.31   0  0.538  6.575  65.2  4.0900   1  296.0  15.3   
1  0.02731   0.0  7.07   0  0.469  6.421  78.9  4.9671   2  242.0  17.8   
2  0.02729   0.0  7.07   0  0.469  7.185  61.1  4.9671   2  242.0  17.8   
3  0.03237   0.0  2.18   0  0.458  6.998  45.8  6.0622   3  222.0  18.7   
4  0.06905   0.0  2.18   0  0.458  7.147  54.2  6.0622   3  222.0  18.7   

       11    12    13  
0  396.90  4.98  24.0  
1  396.90  9.14  21.6  
2  392.83  4.03  34.7  
3  394.63  2.94  33.4  
4  396.90  5.33  36.2  


In [3]:
features = features_data.to_numpy()
labels = df[df.columns[-1]].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(404, 13) (102, 13) (404,) (102,)


In [4]:
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
Y_train = torch.tensor(Y_train, dtype=torch.float).view(-1, 1)
Y_test = torch.tensor(Y_test, dtype=torch.float).view(-1, 1)
print(X_train)
print(X_train.shape, Y_train.shape, X_train.type(), Y_train.type())

tensor([[-0.4186,  3.3717, -1.4455,  ..., -0.2566,  0.4300, -1.0983],
        [-0.4183,  3.3717, -1.0767,  ..., -1.1804,  0.3249, -1.3364],
        [ 0.3401, -0.4872,  1.0150,  ...,  0.8058,  0.3875, -1.3574],
        ...,
        [-0.3955,  0.0487, -0.4762,  ..., -1.5037,  0.3281,  2.4194],
        [-0.4013,  0.3703, -0.6088,  ...,  0.0667,  0.4406, -0.8490],
        [ 2.4633, -0.4872,  1.0150,  ...,  0.8058,  0.4406,  0.9966]])
torch.Size([404, 13]) torch.Size([404, 1]) torch.FloatTensor torch.FloatTensor


In [5]:
model = torch.nn.Sequential(torch.nn.Linear(13, 1))
torch.nn.init.normal_(model[0].weight, mean=0, std=0.1)
torch.nn.init.constant_(model[0].bias, val=0)

datasets = torch.utils.data.TensorDataset(X_train, Y_train)
train_iter = torch.utils.data.DataLoader(datasets, batch_size=10, shuffle=True)
loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [6]:
# Обучение
num_epochs = 1000
for epoch in range(1, num_epochs + 1):
    for x, y in train_iter:
        output = model(x)
        l = loss(output, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

    if epoch % 100 == 1:
        print(f"epoch {epoch}, loss: {l.item():.2f}")

epoch 1, loss: 198.47
epoch 101, loss: 36.75
epoch 201, loss: 10.35
epoch 301, loss: 81.07
epoch 401, loss: 82.69
epoch 501, loss: 12.88
epoch 601, loss: 68.39
epoch 701, loss: 8.85
epoch 801, loss: 9.92
epoch 901, loss: 7.51


In [7]:
# Тест
print(f'\nTest: {loss(model(X_test), Y_test).item()}\n')

# Веса
print(f'Weights: {model[0].weight}, bias: {model[0].bias}')


Test: 17.33361053466797

Weights: Parameter containing:
tensor([[-0.8003,  0.9713, -0.0501,  0.5733, -2.0612,  2.3547,  0.0678, -3.0003,
          2.8213, -2.1549, -2.0479,  0.9963, -4.2588]], requires_grad=True), bias: Parameter containing:
tensor([22.5321], requires_grad=True)
